In [1]:
# 라이브러리 호출
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

In [2]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [3]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'Naver_News_List.pkl',
 'naver_news_samsung.csv',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']

In [4]:
# pkl 파일을 읽고 newsList 생성
newsList = pd.read_pickle(filepath_or_buffer = 'Naver_News_List.pkl')

In [5]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
3652,한국경제TV,"삼성전자, 저전력 특화 '24Gb GDDR7 D램' 개발",https://n.news.naver.com/mnews/article/215/000...,2시간 전
3653,지디넷코리아,"삼성전자, 28일 갤럭시북5 프로 360 국내 출시",https://n.news.naver.com/mnews/article/092/000...,1일 전
3654,노컷뉴스,"삼성전자, AI PC '갤럭시 북5 프로 360' 28일 국내 출시",https://n.news.naver.com/mnews/article/079/000...,1일 전
3655,뉴시스,"삼성전자·SK하이닉스 강세…""TSMC 실적 촉각""[핫스탁]",https://n.news.naver.com/mnews/article/003/001...,2시간 전
3656,세계일보,삼성전자에 발등 찍힌 국민연금 [경제 레이더],https://n.news.naver.com/mnews/article/022/000...,A14면 1단


In [6]:
# newsList의 정보 확인
newsList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   3657 non-null   object
 1   title   3657 non-null   object
 2   nlink   3657 non-null   object
 3   date    3657 non-null   object
dtypes: object(4)
memory usage: 114.4+ KB


In [7]:
# 네이버뉴스 링크로 HTTP 요청을 실행하고 뉴스 본문을 반환하는 함수 생성
def NaverNewsBody(nlink):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열 처리 코드 분기
    if 'https://n.news' in nlink:
        
        # 네이버뉴스 링크로 HTTP 요청 실행
        res = requests.get(url = nlink)
        
        # HTTP 응답 바디 문자열을 bs4.BeautifulSoup 객체로 변환
        soup = bts(markup = res.text, features = 'html.parser')
        
        # soup에서 뉴스 본문을 포함하는 HTML 요소 선택
        item = soup.select_one(selector = '#dic_area, div._article_content')
        
        # item의 텍스트 반환
        # [참고] strip()은 문자열 양 옆에 있는 공백을 모두 제거함
        return item.text.strip()
    
    elif 'https://m' in nlink:
        
        # 'https://m'으로 시작하는 nlink의 일부 문자열 변경한 url 생성
        url = re.sub(pattern = 'https://m.(sports|entertain).naver.com(.+)?(?=/article)', 
                     repl = r'https://api-gw.\1.naver.com/news', 
                     string = newsList['nlink'].iloc[i])
        
        # url로 HTTP 요청 실행
        res = requests.get(url = url)
        
        # JSON 형태의 문자열을 딕셔너리로 변환
        dic = json.loads(s = res.text)
        
        # 뉴스 본문 반환
        return dic['result']['articleInfo']['article']['content']

In [8]:
# 인덱스 설정: 첫 번째 원소
# [참고] 반복문으로 실행할 코드에 인덱스를 추가하면 편리함
i = 0

In [9]:
# 해당 인덱스의 nlink 확인
print(newsList['nlink'].iloc[i])

https://n.news.naver.com/mnews/article/448/0000483378


In [10]:
# 함수 테스트
NaverNewsBody(nlink = newsList['nlink'].iloc[i])

"성전자가 업계 최초로 12나노급 24Gb GDDR7 D램 개발을 완료했다.24Gb GDDR7 D램은 업계 최고 사양을 구현한 제품으로 PC, 게임 콘솔 등 기존 그래픽 D램의 응용처를 넘어 AI 워크스테이션, 데이터센터 등 고성능 제품을 필요로 하는 분야까지 다양하게 활용될 전망이다.이번 제품은 24Gb의 고용량과 400Gbps 이상의 속도를 갖췄고, 전작 대비 용량, 성능, 전력 효율이 모두 향상됐다.삼성전자는 이번 제품에 12나노급 미세 공정을 적용해 동일한 패키지 크기에 셀 집적도를 높였고, 전작 대비 50% 향상된 용량을 구현했다.또 'PAM3 신호방식'을 통해 그래픽 D램 중 업계 최고 속도인 40Gbps를 구현했으며 사용 환경에 따라 최대 42.5Gbps까지의 성능을 자랑한다."

In [11]:
# 경고를 출력하지 않도록 설정
import warnings
warnings.filterwarnings('ignore')

In [12]:
# 반복 실행할 횟수 생성
n = len(newsList)
print(n)

3657


In [14]:
# newsList에 뉴스 본문을 저장할 열 추가
newsList['body'] = np.nan

# 반복문으로 네이버뉴스 본문 수집
for i in tqdm(range(n)):
    
    # 네이버뉴스 링크로 뉴스 본문 수집
    newsList['body'].iloc[i] = NaverNewsBody(nlink = newsList['nlink'].iloc[i])
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/3657 [00:00<?, ?it/s]

In [15]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date,body
3652,한국경제TV,"삼성전자, 저전력 특화 '24Gb GDDR7 D램' 개발",https://n.news.naver.com/mnews/article/215/000...,2시간 전,삼성전자가 업계 최초로 12나노급 '24Gb GDDR7 D램' 개발을 완료했다고 1...
3653,지디넷코리아,"삼성전자, 28일 갤럭시북5 프로 360 국내 출시",https://n.news.naver.com/mnews/article/092/000...,1일 전,코어 울트라 200V 내장 47 TOPS급 NPU로 AI 처리 가속삼성전자가 인텔 ...
3654,노컷뉴스,"삼성전자, AI PC '갤럭시 북5 프로 360' 28일 국내 출시",https://n.news.naver.com/mnews/article/079/000...,1일 전,삼성전자 모델이 강력한 AI PC 경험을 제공하는 '갤럭시 북5 Pro 360'을 ...
3655,뉴시스,"삼성전자·SK하이닉스 강세…""TSMC 실적 촉각""[핫스탁]",https://n.news.naver.com/mnews/article/003/001...,2시간 전,*재판매 및 DB 금지[서울=뉴시스] 박주연 기자 = 삼성전자와 SK하이닉스가 17...
3656,세계일보,삼성전자에 발등 찍힌 국민연금 [경제 레이더],https://n.news.naver.com/mnews/article/022/000...,A14면 1단,전 국민의 노후 ‘뒷배’ 국민연금의 주식 운용 실적에 비상등이 켜졌다. 믿었던 대장...


In [16]:
# 결과를 CSV 파일로 저장
newsList.to_csv('naver_news_samsung.csv', index=False, encoding='utf-8-sig')

### 중복 제거 나중에...